In [2]:
# load packages
import pandas as pd
import requests
import pickle

with open("researchears_unique.pkl", "rb") as f:
    researches_unique = pickle.load(f)

In [2]:
researches_unique[0]

'Alex Pentland'

In [3]:
# Get unique authors ID's

ID_list = []

for author in researches_unique:
    try:
        first, last = author.split(" ")
        response = requests.get(f'https://api.semanticscholar.org/graph/v1/author/search?query={first.lower()}+{last.lower()}')
        #Data = pd.DataFrame.from_dict(response.text)
        Query = response.json()
        for API_value in Query['data']:
            ID_list.append(API_value['authorId'])
    except:
        continue

In [20]:
with open("ID_list.pkl", "wb") as f:
    pickle.dump(ID_list, f)

len(ID_list)

1875

In [3]:
with open("ID_list.pkl", "rb") as f:
    ID_list = pickle.load(f)

In [4]:
# Get Author dataset, Paperdataset and Abstract

AuthorData = pd.DataFrame(columns=['AuthorID', 'Name', 'Aliases', 'CitationCount','Field'])
PaperData = pd.DataFrame(columns=['paperId', 'title', 'abstract', 'year','externalIds','citationCount', 'authors','s2FieldsOfStudy'])
AbstractData = pd.DataFrame(columns=['PaperID', 'Abstract'])

i = 0

for ID in ID_list:
    try:
        response = requests.get(f'https://api.semanticscholar.org/graph/v1/author/{ID}?fields=authorId,name,externalIds,aliases,citationCount,affiliations,papers.paperId,papers.title,papers.year,papers.externalIds,papers.citationCount,papers.abstract,papers.authors,papers.s2FieldsOfStudy')
        Query = response.json()

        AuthorData = pd.concat([AuthorData,
                                pd.DataFrame.from_dict([{'AuthorID': Query['authorId'], 'Name': Query['name'], 'Aliases': Query['aliases'], 'CitationCount': Query['citationCount'], 'Field': None}])])

        #Data = pd.DataFrame.from_dict(Query[])

        Papers = pd.DataFrame(Query['papers'])
        
        PaperData = pd.concat([PaperData, Papers])
    except:
        continue



In [5]:
# Clean Data
AbstractData = PaperData[['paperId', 'abstract']]
PaperData = PaperData.drop(['abstract'], axis=1)

AuthorData.to_excel('AuthorData.xlsx')
PaperData.to_excel('PaperData.xlsx')
AbstractData.to_excel('AbstractData.xlsx')